In [150]:
from codes.utils.file import *
import re
import numpy as np

In [151]:
gt_output_folder= "output"
gt_output_file_name= "1_gt.tre"
true_species_tree_name= "true-species.out.tree"
All_GTs_file_name = "all_gt.tre"
astral_tree_name = "astral.exact.tre"

In [152]:
dataset_path = join_dir("dataset", "Datasets-Biological")

In [153]:
# run avian
avian_path = join_dir(dataset_path, "avian")
# desired_path = join_dir(avian_path, "exon")

aminota_nt_path = join_dir(dataset_path, "aminota-nt")
desired_path = aminota_nt_path

In [154]:
def remove_one(input_str):
    output_str = re.sub(r"1(?=[^\d_])", "", input_str)
    return output_str

def remove_numbers_after_colon(input_str):
    output_str = re.sub(r':[^,\)]+', '', input_str)
    output_str = remove_one(output_str)
    return output_str

def remove_numbers_before_and_after_colon(input_str):
    output_str = re.sub(r'((\d+\.\d+)|\d+):[^,\)]+', '', input_str)
    return output_str

def remove_any_numbers(input_str):
    output_str = re.sub(r'((\d+\.\d+)|\d+)', '', input_str)
    return output_str

In [155]:
def remove_and_create_output_folder(dir_current_path):
    remove_dir_current(dir_current_path)
    create_dir_current(dir_current_path)

In [156]:
# analyze avian(exon)
remove_and_create_output_folder(gt_output_folder)

# read all_gt.tre from exon folder and write to 1_gt.tre and remove number after colon in each gt

gt_path = join_dir(desired_path, All_GTs_file_name)
gt_output_path = join_dir(gt_output_folder, gt_output_file_name)

with open(gt_path, 'r') as f:
    with open(gt_output_path, 'w') as f1:
        for line in f:
            line = remove_numbers_after_colon(line)
            line = remove_any_numbers(line)
            f1.write(line)

# copy astral tree to output folder
astral_tree_path = join_dir(desired_path, astral_tree_name)
astral_tree_output_path = join_dir(gt_output_folder, "input.nwk")
copy_file_current(astral_tree_path, astral_tree_output_path)

In [157]:
def getTaxaSet(t):
    return t.replace('(', '').replace(')', '').split(';')[0].split(':')[0].split(',')

In [158]:
def get_INPUT_DATA():
    OUTGROUP = 'GAL'
    st = open('output/input.nwk')
    st = st.read()
    gts = open('output/1_gt.tre')

    gts = gts.read().split(';\n')[:-1] 

    # print(len(gts))

    # print(st)
    taxaSet = getTaxaSet(st)
    # print(taxaSet)
    matrix = np.zeros((len(taxaSet), len(gts)), dtype=int)


    # print(len(gts))
    for gt_idx, gt in  enumerate(gts) :
        # print(gt)
        cur_taxaSet = getTaxaSet(gt)
        # print(cur_taxaSet)
        for i, tx in enumerate(taxaSet):
            if tx  in cur_taxaSet or tx == OUTGROUP:
                matrix[i][gt_idx] = 1
    with open('output/input.data', 'w', newline='') as f:
        print(matrix.shape[0], matrix.shape[1], file=f)
        for i in range(len(taxaSet)):
            # f.write(matrix[i].tostring(), taxaSet[i])
            print(*matrix[i],taxaSet[i], end='\n', file=f)

In [159]:
get_INPUT_DATA()

In [160]:
def getTerrace():
    # !rm -rf terrace_output
    remove_dir_current("terrace_output")
    # !mkdir -p terrace_output
    create_dir_current("terrace_output")
    
    !./bin/raxml-ng output/input.nwk output/input.data > terrace_output/terrace_output.txt

In [161]:
getTerrace()